In [1]:
import netifaces
# from netifaces import interfaces
import pprint
import re
import winreg as wr

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(netifaces.interfaces())

#判断ip是否为“10”开头
pattern = r'^10'

#定义获取Windows系统中为“10”开头的网卡名及其ip
def get_name_AND_ip():
    
    #获取所有网络接口卡的键值
    id = netifaces.interfaces()
    #存放网卡键值与键值名称的字典
    key_name = {}
    
    try:
        #建立链接注册表，"HKEY_LOCAL_MACHINE"，None表示本地计算机
        reg = wr.ConnectRegistry(None,wr.HKEY_LOCAL_MACHINE)
        # 打开r'SYSTEM\CurrentControlSet\Control\Network\{4d36e972-e325-11ce-bfc1-08002be10318}'，固定的
        reg_key = wr.OpenKey(reg , r'SYSTEM\CurrentControlSet\Control\Network\{4d36e972-e325-11ce-bfc1-08002be10318}')
    except :
        return ('路径出错或者其他问题，请仔细检查')


    for i in id:
        try:
            #尝试读取每一个网卡键值下对应的Name
            reg_subkey = wr.OpenKey(reg_key , i + r'\Connection')
            #如果存在Name，写入key_name字典
            key_name[wr.QueryValueEx(reg_subkey , 'Name')[0]] = i
            # print(wr.QueryValueEx(reg_subkey , 'Name')[0])
        except FileNotFoundError:
            pass
    # print('所有接口信息字典列表： ' + str(key_name) + '\n')
    
      #输出系统所有网卡键值及其网卡名
#     for name, key in key_name.items():
#         print(name)
#         try:
#             pp.pprint(netifaces.ifaddresses(key)[netifaces.AF_INET][0]['addr'])
#         except KeyError:
#             print("NULL")
#             pass
#         print("---------------------------------")

    #输出系统“10”开头ip及其对应网卡名
    for name, key in key_name.items(): 
        try:
            ip = netifaces.ifaddresses(key)[netifaces.AF_INET][0]['addr']
            if re.search(pattern, ip):
                break
            else:
                name = 'No matching IP was found'
                ip = 'NULL'         
        except KeyError:
            ip = 'NULL'
            pass  
        
    
    #将找到的网卡名及其ip输出到对应位置    
    data =[[name],[ip]]
    with open("data.txt","w") as f:                                                 #设置文件对象
        for i in data:                                                                 #对于双层列表中的数据
            i = str(i).strip('[').strip(']').replace(',','').replace('\'','')+'\n'     #将其中每一个列表规范化成字符串
            f.write(i)    
        f.close()
    
        
        
    result = (name, ip)
    return result


a = get_name_AND_ip()
print(f"本次需更改的网卡名称：{a}")

本次需更改的网卡名称：('No matching IP was found', 'NULL')
